<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E6%96%B0%E5%B0%8F%E8%AF%B4%E6%8A%BD%E5%8F%96_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境

本脚本只需要在CPU环境运行

如果你不在colab运行，你需要一个openai不封禁的ip

In [ ]:
!pip -q install langchain openai google-search-results tiktoken
!pip -q install kor markdownify


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00


# 新小说提取

Implemented by 李鲁鲁

检查下列变量是否被正确复制

- [ ] openai的key（不要把自己的key 上传到github！）
- [ ] 指定下载小说 wget后面的网址
- [ ] 指定保存位置 save_folder

In [ ]:

import os

key = 'sk-VvF4'
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [ ]:
!wget https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E5%B0%84%E9%9B%95%E8%8B%B1%E9%9B%84%E4%BC%A0.txt

input_name = '/content/射雕英雄传.txt'

--2023-09-01 01:48:21--  https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E5%B0%84%E9%9B%95%E8%8B%B1%E9%9B%84%E4%BC%A0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2795446 (2.7M) [text/plain]
Saving to: ‘射雕英雄传.txt’

射雕英雄传.txt      100%[===================>]   2.67M  --.-KB/s    in 0.05s   

2023-09-01 01:48:22 (56.2 MB/s) - ‘射雕英雄传.txt’ saved [2795446/2795446]



In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

import os

save_folder = "/content/drive/MyDrive/StoryGPT/shediaoyingxiongzhuan_extract"

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
else:
    print('注意，文件夹',save_folder,'已经存在')


Mounted at /content/drive


# 下载和切片天龙八部

这里你要给小说切一下章节（最好）

如果你非常非常懒

可以使用

```python
chapters = [raw_text]
```

In [ ]:
import re

raw_text = open(input_name, encoding='utf-8').read()

# chapter_pattern = r'第\d+章[^\n]+\n'

# chapters = raw_text.split(chapter_pattern)

# print(len(chapters))

我觉得这段代码现在比正则表达式好用

当然这个代码是针对 第xx章 xxxx 这种格式的

如果有的时候你是 ---第xx章--

你要改一下那个startswith那个 或者把前面的特殊符号给strip掉

In [ ]:
chapters = []
chapter_contents = []

for line in raw_text.split('\n'):
    Flag = False
    if line.strip().startswith('第'):
        # 遇到章节标题,将之前章节内容添加到结果列表

        head = line.strip()
        # print(head)
        head = head[:min(10,len(head))]
        if head.find('章',1)>0:
            # print(head)
            Flag = True

    if Flag:
        if chapter_contents:
            chapters.append('\n'.join(chapter_contents))
            chapter_contents = []
        # 记录当前章节标题
        # chapters.append(line)
    else:
        # 累积章节内容
        chapter_contents.append(line)

# 添加最后一个章节内容
if chapter_contents:
    chapters.append('\n'.join(chapter_contents))

print(len(chapters))

40


看一看提取出来的章节对不对

In [ ]:
for i,ch in enumerate(chapters):
    print(i,ch[:10])

0 
钱塘江浩浩江水，日
1 
颜烈跨出房门，只见
2 
寺里僧众见焦木圆寂
3 
完颜洪熙笑道：“好
4 
一行人下得山来，走
5 
帐中六怪低声计议。
6 
江南六怪与郭靖晓行
7 
王处一脚步好快，不
8 
郭黄二人来到赵王府
9 
完颜康斗然见到杨铁
10 
沙通天见师弟危殆，
11 
黄蓉正要将鸡撕开，
12 
黄蓉回到客店安睡，
13 
只见五男一女，走进
14 
陆冠英扶起完颜康，
15 
郭、黄二人自程府出
16 
周伯通道：“你道是
17 
郭靖循着蛇声走去，
18 
洪七公万万想不到这
19 
洪七公与郭靖见欧阳
20 
欧阳锋只感身上炙热
21 
黄蓉见欧阳锋拖泥带
22 
洪七公、周伯通、郭
23 
黄蓉向外走了两步，
24 
黄药师仰天一笑，说
25 
黄药师心想不明不白
26 
两人正闹间，楼梯声
27 
此时鲁有脚已经醒转
28 
郭靖在雕背连声呼叫
29 
两人顺着山路向前走
30 
一灯大师低低叹了口
31 
穆念慈右手让黄蓉握
32 
郭靖与黄蓉此刻心意
33 
黄蓉飞舞正急，忽然
34 
船靠岸边，走上二三
35 
黄蓉幽幽的道：“欧
36 
这一日郭靖驻军那密
37 
郭靖解下长衣，执住
38 
郭靖纵马急驰数日，
39 
欧阳锋冷冷的道：“


In [ ]:
print(len(chapters))
print(chapters[0][:100])

40

钱塘江浩浩江水，日日夜夜无穷无休的从临安牛家村边绕过，东流入海。

    江畔一排数十株乌柏树，叶子似火烧般红，正是八月天时。村前村后的野草刚起始变黄，一抹斜阳映照之下，更增了几分萧索。两株大松树


In [ ]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

In [ ]:
#@title  定义divide函数，用来切分超长文本
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s):
    left, right = divide_str(s)

    if right != '':
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]


In [ ]:
#@title 以1500 token为限，切分chunk，输出总chunk数量

max_token_len = 1500
chunk_text = []


for chapter in chapters:

    split_text = chapter.split('\n')

    curr_len = 0
    curr_chunk = ''

    tmp = []

    for line in split_text:
        line_len = len(enc.encode( line ))

        if line_len <= max_token_len - 5:
            tmp.append(line)
        else:
            # print('divide line with length = ', line_len)
            path = [line]
            tmp_res = []

            while path:
                my_str = path.pop()
                left, right = strong_divide(my_str)

                len_left = len(enc.encode( left ))
                len_right = len(enc.encode( right ))

                if len_left > max_token_len - 15:
                    path.append(left)
                else:
                    tmp_res.append(left)

                if len_right > max_token_len - 15:
                    path.append(right)
                else:
                    tmp_res.append(right)

            for line in tmp_res:
                tmp.append(line)

    split_text = tmp

    for line in split_text:
        line_len = len(enc.encode( line ))

        if line_len > max_token_len:
            print('warning line_len = ', line_len)

        if curr_len + line_len <= max_token_len:
            curr_chunk += line
            curr_chunk += '\n'
            curr_len += line_len
            curr_len += 1
        else:
            chunk_text.append(curr_chunk)
            curr_chunk = line
            curr_len = line_len

    if curr_chunk:
        chunk_text.append(curr_chunk)

    # break

print(len(chunk_text))

929


# 编辑新的Kor

新的Kor要简单很多，我们准备不再提取，动作。只保留对话

这里的Kor建议你调一调 当然如果很懒，也可以不调了

In [ ]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.llms import OpenAI
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [ ]:
print(chunk_text[100])

    张翠山沉吟道：“这梅花小镖用左手发射？少林派门下怎地出现了道人，莫非也是乔装的？”那少女微笑道：“道士扮和尚须剃光头，和尚扮道士却容易得多，戴顶道冠便成。”张翠山点了点头。那少女道：“我心知此事不妙，但那瘦子我尚自抵敌不过，那道人似乎更厉害得多，何况他们共有六人？这可没了计较。”张翠山张口欲言，但终于忍住了。那少女道：“我猜你是想问：‘干么不上武当山来跟我们说明？’是不是？我可不能上武当山啊，倘若我自己能出面，又何必委托都大锦走这趟镖呢？我徬徨无计，在道上闷走，恰好撞到你跟都大锦他们说话。后来见你去找寻俞三侠，我想武当七侠正主儿已接上了手，不用我再凑热闹，凭我这点微末本领，也帮不了甚么忙。那时我急于解毒，便即东还，不知俞三侠后来怎样了？”张翠山当下说了俞岱岩受人毒害的情状。那少女长叹一声，睫毛微微颤动，说道：“但愿俞三侠吉人天相，终能治愈，否则……否则……”张翠山听她语气诚恳，心下感激，说道：“多谢姑娘好心。”说着眼眶微湿。那少女摇了摇头，说道：“我回到江南，叫人一看这梅花镖，有人识得是少林派的独门暗器，说道除非是发暗器之人的本门解药，否则毒性难除。临安府除了龙门镖局，还有谁是少林派？于是我夜入镖局，要逼他们给解药，岂知他们不但不给，还埋伏下了人马，我一进门便对我猛下毒手。”张翠山“嗯”了一声，沉吟道：“你说故意安排，教他们认作是我？”那少女脸有腼腆之色，低下了头，轻轻的道：“我见你到衣铺去买了这套衣巾，觉得穿戴起来很是……很是好看，于是我跟着也头了一套。”张翠山道：“这便是了。只是你一出手便连杀数十人，未免过于狠辣，镖局中的人跟你又没怨仇。”那少女沉下脸来，冷笑道：“你要教训我么？我活了一十九岁，倒还没听人教训过呢。张五侠大仁大义，这就请罢。我这般心狠手辣之辈，原没盼望跟你结交。”



In [ ]:
#@title 仅仅抽取dialogue的部分

schema = Object(
    id="script",
    description="Extract Dialogue in order From Novel, ignore the non-dialogue parts",
    attributes=[
        Text(
            id="role",
            description="The character who is speaking, use context to predict the name of the role.",
        ),
        Text(
            id="dialogue",
            description="The dialogue spoken by the characters in the sentence",
        ),
    ],
    examples=[
        (
            '''``村民中走出一个二十来岁的人汉，说道：“张先生，你可是从北方来吗”

    张十五见他身材魁梧，浓眉大眼，便道：“正是。”那大汉道：“小弟作东，请先生去饮上三杯如何”张十五大喜，说道：“素不相识，怎敢叨扰”

    那大汉笑道：“喝上三怀，那便相识了。我姓郭，名叫郭啸天。”``''',
            [
                {"role": "郭啸天", "dialogue": "张先生，你可是从北方来吗"},
                {"role": "张十五","dialogue": "正是。"},
                {"role": "郭啸天", "dialogue": "小弟作东，请先生去饮上三杯如何"},
                {"role": "张十五","dialogue": "素不相识，怎敢叨扰"},
                {"role": "郭啸天", "dialogue": "喝上三怀，那便相识了。我姓郭，名叫郭啸天。"}
            ],
        )
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(llm, schema)
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

script: Array<{ // Extract Dialogue in order From Novel, ignore the non-dialogue parts
 role: string // The character who is speaking, use context to predict the name of the role.
 dialogue: string // The dialogue spoken by the characters in the sentence
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: ``村民中走出一个二十来岁的人汉，说道：“张先生，你可是从北方来吗”

    张十五见他身材魁梧，浓眉大眼，便道：“正是。”那大汉道：“小弟作东，请先生去饮上三杯如何”张十五大喜，说道：“素不相识，怎敢叨扰”

    那大汉笑道：“喝上三怀，那便相识了。我姓郭，名叫郭啸天。”``
Output: role|dialogue
郭啸天|张先生，你可是从北方来吗
张十五|正是。
郭啸天

In [ ]:
i = 250
response = chain.run( f"``{chunk_text[i]}``" )["data"]

In [ ]:
print(response)
# print(chunk_text[i])

{'script': [{'role': '王处一', 'dialogue': '你武功可比你爹爹强得多呀，那是怎么回事'}, {'role': '穆念慈', 'dialogue': '晚辈十三岁那年，曾遇到一位异人。他指点了我三天武功，可惜我生性愚鲁，没能学到甚么。'}, {'role': '王处一', 'dialogue': '他只教你三天，你就能胜过你爹爹。这位高人是谁'}, {'role': '穆念慈', 'dialogue': '不是晚辈胆敢隐瞒道长，实是我曾立过誓，不能说他的名号。'}, {'role': '王处一', 'dialogue': '丘师哥，你教完颜康教了有八九年吧'}, {'role': '丘处机', 'dialogue': '整整九年零六个月，唉，想不到这小子如此混蛋。'}, {'role': '王处一', 'dialogue': '这倒奇了'}, {'role': '丘处机', 'dialogue': '怎么'}, {'role': '柯镇恶', 'dialogue': '丘道长，你怎么找到杨大哥的后裔'}, {'role': '丘处机', 'dialogue': '说来也真凑巧。自从贫道和各位订了约会之后，到处探访郭杨两家的消息，数年之中，音讯全无，但总不死心，这年又到临安府牛家村去查访，恰好见到有几名公差到杨大哥的旧居来搬东西。贫道跟在他们背后，偷听他们说话，这几个人来头不小，竟是大金国赵王府的亲兵，奉命专程来取杨家旧居中一切家私物品，说是破凳烂椅，铁枪犁头，一件不许缺少。'}, {'role': '丘处机', 'dialogue': '贫道晚上夜探王府，要瞧瞧赵王万里迢迢的搬运这些破烂物事，到底是何用意。一探之后，不禁又是气愤，又是难受，原来杨兄弟的妻子包氏已贵为王妃。贫道大怒之下，本待将她一剑杀却，却见她居于砖房小屋之中，抚摸杨兄弟铁枪，终夜哀哭；心想她倒也不忘故夫，并非全无情义，这才饶了她性命。后来查知那小王子原来是杨兄弟的骨血，隔了数年，待他年纪稍长，贫道就起始传他武艺。'}, {'role': '柯镇恶', 'dialogue': '那小子是一直不知自己的身世的了'}, {'role': '丘处机', 'dialogue': '贫道也曾试过他几次口风，见他贪恋富贵，不是性情中人，是以始终不曾点被。几次教诲他为人立身之道，这

理论上这个抽取出来是这样的

```
role|dialogue
干光豪|自今而后,咱二人再也不分什么东宗西宗啦。我俩东宗西宗联姻,合为一体......
葛师妹|别......别这样。
干光豪|你依了我,若是我日后负心,就掉在这水里,变个大忘八。
葛师妹|你做忘八,可不是骂我不规矩吗?
干光豪|什么人?
干光豪|葛师妹,你拦住了那边山口!
干光豪|前面是本派禁地,任何外人不得擅入。你再向前数丈,干犯禁忌,可叫你死葬身之地。
段誉|我要性命,这才逃走......
段誉|啊哟!
```

但是我遇到了latin error 所以下次在继续调试这部分

让我们来看看摘要的部分

# Bullet形式摘要抽取

In [ ]:
system_prompt = """
Summarize the key points of the following text in a concise way, using bullet points.
"""

q_example = """###
Text:
洪七公、周伯通、郭靖、黄蓉四人乘了小船，向西驶往陆地。郭靖坐在船尾扳桨，黄蓉不住向周伯通详问骑鲨游海之事，周伯通兴起，当场就要设法捕捉鲨鱼，与黄蓉大玩一场。
郭靖见师父脸色不对，问道：“你老人家觉得怎样”洪七公不答，气喘连连，声息粗重。他被欧阳锋以“透骨打穴法”点中之后，穴道虽已解开，内伤却又加深了一层。黄蓉喂他服了几颗九花玉露丸，痛楚稍减，气喘仍是甚急。
老顽童不顾别人死活，仍是嚷着要下海捉鱼，黄蓉却已知不妥，向他连使眼色，要他安安静静的，别吵得洪七公心烦。周伯通并不理会，只闹个不休。黄蓉皱眉道：“你要捉鲨鱼，又没饵引得鱼来，吵些甚么”

Summarize in BULLET POINTS form:
"""

a_example = """
- 洪七公等四人乘船西行,洪七公因受内伤加重而气喘不止
- 周伯通要捉鲨鱼玩,被黄蓉阻止以免掀翻小船
"""



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [ ]:
messages = [SystemMessage( content = system_prompt),
            HumanMessage( content = q_example),
            AIMessage( content = a_example)]

sel_id = 300

new_input = f"""###
Text:
{chunk_text[sel_id]}

Summarize in BULLET POINTS form:"""

messages.append( HumanMessage(content = new_input) )

response = llm( messages )

print(response.content)

- 完颜康和陆冠英进行比武较量
- 陆冠英使用九阴白骨爪和全真派拳法对抗完颜康的手爪功夫
- 陆冠英展示了他的外家功夫，特别是腿上的技巧
- 完颜康在战斗中受伤，但最终使用绝招“怀心腿”将陆冠英击败
- 陆冠英被完颜康推倒，撞向陆庄主，引发了一场争斗


理论上输出是这样的


- 段誉和朱丹臣讨论诗词,表达友谊
- 木婉清不懂诗词,觉得两人说书呆子话
- 朱丹臣礼貌对木婉清,想请两人回段家
- 段誉担心鳄神,同意回家,让朱丹臣护送
- 木婉清说朱丹臣话太多,自己是山野女子不懂
- 朱丹臣表示会简化自己的文词,不让木婉清觉得难懂


In [ ]:
print(save_folder)

/content/drive/MyDrive/StoryGPT/shediaoyingxiongzhuan_extract


Mounted at /content/drive


In [ ]:
import os
import json
from tqdm import tqdm

# save_folder = "/content/drive/MyDrive/GPTData/weixiaobao_extract"

for i in tqdm(range(len(chunk_text))):
    save_name = os.path.join(save_folder, f"{i}_dialogue.txt")

    if not os.path.exists(save_name) or os.path.getsize(save_name) < 5:
        if os.path.exists(save_name):
            print('re-generate dialogue id = ', i)
        query_text = f"``{chunk_text[i]}``"
        dialogue_response = chain.run( query_text )["data"]

        with open(save_name, 'w', encoding='utf-8') as f:
            if 'script' not in dialogue_response:
                print('Error: response does not contain key "script"')
            else:
                for chat in dialogue_response['script']:
                    json_str = json.dumps(chat, ensure_ascii=False)
                    f.write(json_str+"\n")

    save_name_sum = os.path.join(save_folder, f"{i}_sum.txt")

    if not os.path.exists(save_name_sum) or os.path.getsize(save_name) < 5:
        if os.path.exists(save_name_sum):
            print('re-summarize id = ',i )
        #dealing with summarize
        messages = [SystemMessage( content = system_prompt),
                HumanMessage( content = q_example),
                AIMessage( content = a_example)]

        new_input = f"###\nText:\n{chunk_text[ i ]}\nSummarize in BULLET POINTS form:"

        messages.append( HumanMessage(content = new_input) )

        summarize_response = llm( messages ).content

        with open(save_name_sum, 'w', encoding='utf-8') as f:
            f.write( summarize_response )

    raw_text_save_name = os.path.join(save_folder, f"{i}_raw.txt")

    if not os.path.exists(raw_text_save_name) or os.path.getsize(raw_text_save_name) < 5:
        with open(raw_text_save_name, 'w', encoding='utf-8') as f:
            f.write( chunk_text[i] )

    # if i >5:
    #     break


  6%|▌         | 57/929 [24:58<7:11:13, 29.67s/it]

In [ ]:
print(chunk_text[984])

    游坦之乘着众人扰攘之际，和全冠清低声商议了一阵，又朗声道：“我大宋国步艰危，江湖同道却又不能齐心合力，以至时受番帮欺压。因此丐帮主张立一位武林盟主，大伙儿听奉号令，有什么大事发生，便不致乱成一团了。玄慈方丈，你赞不赞成？”    玄慈缓缓地道：“庄帮主的话，倒也言之成理。但老衲有一事不解，却要请教。”游坦之道：“什么事？”玄慈道：“庄帮主已拜丁先生为师，算是星宿派门人了，是也不是？”游坦之道：“这个……这是我自己的事，与你无关。”玄慈道：“星宿派乃西域门派，非我大宋武林同道。我大宋立不立武林盟主，可与星宿派无涉。就算中原武林同道要推举一位盟主，以便统筹事功，阁下是星宿派门人，却也不便参与了。”
    众英雄纷纷说道：“不错！”“少林方丈之言甚是。”“你是番邦门派的走狗奴才，怎可妄想做我中原武林的盟主？。”
    游坦之无言可答，向丁春秋望望，又向全冠清瞧瞧，盼望他们出言解围。
    丁春秋咳嗽一声，说道：“少林方丈言之差矣！老夫乃山东曲阜人氏，生于圣人之邦，星宿派乃老夫一手创建，怎能说是西域番邦的门派？星宿派虽居处西域，那只不过是夫夫暂时隐居之地。你说星宿派是番邦门派，那么孔夫子也是番邦人氏了，可笑啊可笑！说到西域番邦，少林武功源于天竺达摩祖师，连佛教也是西域番邦之物，我看少林派才是西域的门派呢！”此言一出，玄慈和群雄都感不易抗辩。
    全冠清朗声道：“天下武功，源流难考。西域武功传于中土者有之，中土武功传于西域者亦有之。我帮庄帮主乃中土人氏，丐帮素为中原门派，他自然是中原武林的领袖人物。玄慈方丈，今日之事，当以武功强弱定胜负，不以言辞舌辩定输赢。丐帮与少林派到底谁强谁弱，只须你们两位首领出手较量，高下立判，否则便是说上半天，又有何益？倘若你有自知之明，不是敝帮庄帮主的敌手，只须甘拜下风，推戴我庄帮主为武林盟主，倒也不是非出手不可的。”这几句话，显然认定玄慈是明知不敌，胆怯推诿。
    玄慈向前走了几步，说道：“庄帮主，你既非要老衲出手不可，老衲若再顾念贵帮和敝帮数百年的交情，坚不肯允，倒是对贵帮不敬了。”眼光向群雄缓缓掠过，朗声道：“天下英雄，今日人人亲眼目睹，我少林派决无与丐帮争雄斗胜之意，实是丐帮帮主步步见逼，老衲退无可退，避无可避。”
    群雄纷纷说道：“不错，咱们都是见证，少林派并无丝毫理亏之处。”
    游坦之只是挂念着阿紫的安